In [36]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
import os
import joblib
import numpy as np
import pandas as pd

In [3]:
ws = Workspace.from_config()
# choose a name for experiment
experiment_name = 'labor-prediction-experiment'
experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RJ9HERHE4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [32]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

datastore_name='workspaceblobstore'
datastore=Datastore.get(ws,datastore_name)

datastore_path = [(datastore, 'UI/01-03-2021_103243_UTC/LaborPredictionData6.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

ds = ds.take(3000).to_pandas_dataframe()

In [17]:
ds.take(3000).to_pandas_dataframe()

,heightwholenum,framemat,framefunc,widthWholeNum,lift,itemset,locationtype,leadmechanic,TotalInstallationTime,LongCarry
0,18,VINYL,TOP SH,31,67,VINYL BEADS,9,228,30,NULL
1,55,alum,FIXED,27,21,wrap around rubber,1,211,30,NULL
2,34,ALUMINUM,FLUSH GLAZE,73,51,PUSH RUBBER,2,6,25,NULL
3,25,ALUMINUM,FLUSH GLAZE,66,9,PUSH RUBBER,2,6,40,NULL
4,58,ALUMINUM,FLUSH GLAZE,50,25,None,2,6,25,NULL
...,...,...,...,...,...,...,...,...,...,...
2155,92,ALUMINUM,FLUSH GLAZE,20,2,None,1,6,60,NULL
2156,15,ALUMINUM,None,46,6,SNAP BEADS,1,6,35,NULL
2157,31,None,BOTTOM SH,24,29,WRAP AROUND RUBBER,1,90,35,NULL
2158,25,None,None,41,3,None,6,6,65,NULL


In [41]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8)

y_train=y_train.reshape(1728, 1)
train = np.concatenate((x_train,y_train), axis=1)

In [44]:
# TODO: Put your automl settings here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    training_data=train,
    label_column_name='y',
    n_cross_validations=10)

In [45]:
#TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Package name/Required version/Installed version
azureml-core/pyopenssl<20.0.0/pyOpenSSL 20.0.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nPackage name/Required version/Installed version\nazureml-core/pyopenssl<20.0.0/pyOpenSSL 20.0.0",
        "target": "azureml-core",
        "inner_error": {
            "code": "NotSupported",
     

In [ ]:
RunDetails(remote_run).show()

In [ ]:
joblib.dump(fitted_mlmodel, \"automlmodel.joblib\")